In [ ]:
%pip install llama-index-readers-file -q


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nest_asyncio

nest_asyncio.apply()

### Custom handlers

In [ ]:
from llama_index.core.instrumentation.event_handlers import BaseEventHandler
from llama_index.core.instrumentation.span_handlers import SimpleSpanHandler

In [ ]:
class MyEventHandler(BaseEventHandler):
    @classmethod
    def class_name(cls) -> str:
        """Class name."""
        return "MyEventHandler"

    def handle(self, event) -> None:
        """Logic for handling event."""
        print(event.class_name())
        with open("log.txt", "a") as f:
            f.write(str(event))
            f.write("\n")


# class MyQueryEndEventHandler(BaseEventHandler):
#     @classmethod
#     def class_name(cls) -> str:
#         """Class name."""
#         return "MyQueryEndEventHandler"

#     def handle(self, event) -> None:
#         """Logic for handling event."""
#         if isinstance(event, QueryEndEvent):
#             print(self.class_name())
#             with open("log.txt", "a") as f:
#                 f.write(str(event))
#                 f.write("\n")

### Dispatcher


In [ ]:
import llama_index.core.instrumentation as instrument

dispatcher = instrument.get_dispatcher()  # modify root dispatcher

In [ ]:
dispatcher.add_event_handler(MyEventHandler())
dispatcher.span_handler = SimpleSpanHandler()

In [ ]:
dispatcher

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

In [ ]:
qe_dispatcher = instrument.get_dispatcher("llama_index.core.base.query_engine")

In [ ]:
qe_dispatcher

Dispatcher(name='llama_index.core.base.query_engine', event_handlers=[], span_handler=NullSpanHandler(open_spans={}, current_span_id=None), parent_name='root', manager=<llama_index.core.instrumentation.dispatcher.Manager object at 0x109146e60>, root_name='root', propagate=True)

In [ ]:
qe_dispatcher.parent

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

In [ ]:
qe_dispatcher.root

Dispatcher(name='root', event_handlers=[NullEventHandler(), MyEventHandler()], span_handler=SimpleSpanHandler(open_spans={}, current_span_id=None, completed_spans=[]), parent_name='', manager=None, root_name='root', propagate=False)

### Test It Out

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

documents = SimpleDirectoryReader(input_dir="./data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
query_result = query_engine.query("Who is Paul?")

In [ ]:
dispatcher.span_handler.completed_spans

[]

In [ ]:
query_result = await query_engine.aquery("Who is Paul?")

QueryStartEvent
RetrievalStartEvent
RetrievalEndEvent
SynthesizeStartEvent
GetResponseStartEvent
LLMPredictStartEvent
LLMPredictEndEvent
GetResponseEndEvent
SynthesizeEndEvent
QueryEndEvent


In [ ]:
query_engine.dispatcher.span_handler.completed_spans

[SimpleSpan(id_='BaseRetriever.retrieve-94735787-1cd9-4a18-af8d-779e6df326b6', parent_id='BaseQueryEngine.query-55813549-6757-44c4-b97d-507ef0ec2c4a', start_time=datetime.datetime(2024, 3, 4, 15, 25, 25, 308791), end_time=datetime.datetime(2024, 3, 4, 15, 25, 25, 447625), duration=0.138834),
 SimpleSpan(id_='LLM.predict-2cb262a6-9cc8-457f-b930-5fd85aac1f25', parent_id='Refine.get_response-64681daf-915c-45b2-97e8-763e32d59886', start_time=datetime.datetime(2024, 3, 4, 15, 25, 25, 456168), end_time=datetime.datetime(2024, 3, 4, 15, 25, 27, 16449), duration=1.560281),
 SimpleSpan(id_='Refine.get_response-64681daf-915c-45b2-97e8-763e32d59886', parent_id='CompactAndRefine.get_response-e809cf8d-0eab-4e69-8534-0af8308eef48', start_time=datetime.datetime(2024, 3, 4, 15, 25, 25, 452438), end_time=datetime.datetime(2024, 3, 4, 15, 25, 27, 16700), duration=1.564262),
 SimpleSpan(id_='CompactAndRefine.get_response-e809cf8d-0eab-4e69-8534-0af8308eef48', parent_id='BaseSynthesizer.synthesize-1f74277